In [ ]:
!pip3 install -U sentence-transformers
!pip3 install -U huggingface_hub 

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sentence_transformers import SentenceTransformer, util

In [3]:
# read twitter user names and ids as dict
# the id as dict key and the mapped name as dict value

twitter_ids_df=pd.read_csv('./twitter_id.txt',header=0)

twitter_ids_df=twitter_ids_df.dropna(subset=['twitter_userid'])

name_id_dict=pd.Series(twitter_ids_df.name.values, index=twitter_ids_df.twitter_userid).to_dict()

In [4]:
# name_id_dict

In [6]:
embedded_vecs=pd.read_csv('./preprocess/twitter_embedded_cevs.csv',header=0)

In [7]:
# display(embedded_vecs)

In [8]:
def similarity_calc(embedded_vecs, user_id) -> zip:
    """
    """
    try:
        if int(user_id) in embedded_vecs['user_id'].values: 
            user_df = embedded_vecs[embedded_vecs['user_id'] == user_id]
        
        # Calculate similarity based on cosine
        cos_sim_results = np.squeeze(
            util.cos_sim(user_df.iloc[:, 1:].values.astype(np.float32), embedded_vecs.iloc[:, 1:].values.astype(np.float32)).numpy())

        top_user_dict = zip(embedded_vecs.iloc[:, 0], cos_sim_results)

    except Exception as e:

        print(e)

    else:
        return top_user_dict

In [9]:
# calculate the top 10 most similar accounts

def top_10_accounts(embedded_vecs,user_id):

    top_user_dict = similarity_calc(embedded_vecs=embedded_vecs,user_id=user_id) 

    # sort the IDs with similarity scores
    results = sorted(top_user_dict, key=lambda item: item[1], reverse=True)[1:11]

    result_df = pd.DataFrame(results, columns=['twitter_id', 'similarity'])

    ids = list(map(int, list(name_id_dict.keys())))

    for idx, row in result_df.iterrows():
        if row['twitter_id'] in ids:
            result_df.at[idx, 'user_name'] = name_id_dict[float(row['twitter_id'])]
        
    return result_df

In [10]:
user_id=27260086
result_df=top_10_accounts(embedded_vecs=embedded_vecs,user_id=user_id)

In [11]:
result_df

,twitter_id,similarity,user_name
0,40908929,0.969320,Usher Raymond IV
1,79650494,0.945055,Cody Simpson
2,23561980,0.943971,Kendrick Lamar
3,22548403,0.941952,Tom Felton
4,27195114,0.941063,Drizzy
5,119509520,0.938085,Chris Brown
6,25421094,0.932922,AKON
7,264107729,0.932102,5 Seconds of Summer
8,255388236,0.930893,The Weeknd
9,33597201,0.929589,Ashley Benson
